In [1]:
from tdmclient import ClientAsync

from global_navigation import GlobalNavigation

client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 6ca1ca45-4524-41eb-8cc4-fb023f64b9d0

In [2]:
from Vision import Vision, show_many_img
import time
from motion_control import MotionControl
from robot_api import Thymio
from global_navigation import GlobalNavigation
import numpy as np

In [3]:
#create vision module object
visio = Vision()
visio.begin()

Caméra initialisée sur le port 4


In [4]:

global_nav = GlobalNavigation()
Thymio = Thymio(node, client)
motion_control = MotionControl(Thymio)

thymio, goal, obstacles = [],[],[]

while not (thymio and goal and obstacles):
    _thymio, _goal, _obstacles = visio.get_thymio_goal_and_obstacles()
    time.sleep(0.2)
    if _thymio : 
        thymio = _thymio
    if _goal :
        goal = _goal
    if _obstacles :
        obstacles = _obstacles

print("everything ok to start")

try:
    while True:
        _thymio, _goal, _obstacles = visio.get_thymio_goal_and_obstacles()
        if _thymio : 
            thymio = _thymio
        if _goal :
            goal = _goal
        if _obstacles :
            obstacles = _obstacles

        path, _, _ = global_nav.dijkstra(thymio, goal, obstacles)
        visio.analysis.path = path
        #path.pop(0)
        goal_point = path[1]
        print("liste des objectifs", path)
        motion_control.obstacle_avoidance()
        if motion_control.path_tracking(thymio, goal_point):
            print("objectif suivant atteint")
            path.pop(0)
            if len(path) == 1:
                motion_control.stop_motor()
                break
        #thymio, goal, obstacles = visio.get_thymio_goal_and_obstacles()
        time.sleep(0.5)
except KeyboardInterrupt:
    print("Stop the program")
finally:
    visio.stop()
    print("Program finishes")

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/epuck/.pyenv/versions/3.10.13/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/epuck/Documents/MoBot/Exercises/Projet/MICRO452_Mobile_Robots/Vision.py", line 509, in run
    self.vision_module.julien_main(frame)
  File "/home/epuck/Documents/MoBot/Exercises/Projet/MICRO452_Mobile_Robots/Vision.py", line 477, in julien_main
    top_view_img, four_point_matrix = self.four_point_transform(img,corners)
  File "/home/epuck/Documents/MoBot/Exercises/Projet/MICRO452_Mobile_Robots/Vision.py", line 325, in four_point_transform
    M = cv2.getPerspectiveTransform(pts.astype('float32'), dst)
cv2.error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/imgwarp.cpp:3624: error: (-215:Assertion failed) src.checkVector(2, CV_32F) == 4 && dst.checkVector(2, CV_32F) == 4 in function 'getPerspectiveTransform'



KeyboardInterrupt: 

In [ ]:
#path.pop(0)
#goal_point = path[1]#

#try:
#    while True:
#        thymio, goal, obstacles = visio.get_thymio_goal_and_obstacles()
#        path, _, _ = global_nav.dijkstra(thymio, goal, obstacles)
#        visio.analysis.path = path
#        goal_point = path[1]
#        motion_control.obstacle_avoidance()
#        if motion_control.path_tracking(thymio, goal_point):
#            path.pop(0)
#            if not path:
#                motion_control.stop_motor()
#                break
#            else:
#                goal_point = path[0]
#        thymio, goal, obstacles = visio.get_thymio_goal_and_obstacles()
#        time.sleep(0.5)
#except KeyboardInterrupt:
#    print("Stop the program")
#finally:
#    visio.stop()
#    print("Program finishes")

In [ ]:
motion_control.set_motor_speed(0, 0)
#test

NameError: name 'motion_control' is not defined